In [1]:
!python -m spacy download pt_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [2]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

# importando o modelo de arquitetura cbow treinado 
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_cbow.txt")

# importando o modelo de arquitetura Skip Gram treinado 
w2v_modelo_sg  = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_skipgram.txt")

# importando os dados de treino e teste
artigo_treino = pd.read_csv("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/treino.csv")
artigo_teste =  pd.read_csv("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/teste.csv")


In [3]:
# Como já vamos utilizar apenas os tokens no treinamento, existem processos realizados
# pelo spacy.load que podem ser descartados como ner=ReconhecimendoDeEntidadesNomeadas
nlp = spacy.load("pt_core_news_sm", disable = ["parser", "ner", "tagger", "textcat"])

In [4]:
# Definindo a função que transforma frases em lista de palavras importantes

def tokenizador(texto):
  doc = nlp(texto) # faz o doc da frase
  tokens_validos = []
  for token in doc: 
    valido = not token.is_stop and token.is_alpha # Se a palavra não é stopword, colocamos na lista vazia
    if valido:
      tokens_validos.append(token.text.lower())
  
  return tokens_validos

In [5]:
texto = "Sonho meu, vai buscar quem mora longe, sonho meu!"

tokens = tokenizador(texto)
tokens

['sonho', 'buscar', 'mora', 'sonho']

In [6]:
# definindo a função que vai transformar as frases em vetores

def combinacao_de_vetores_por_soma(palavras, modelo): # recebe lista de palavras e modelo
  vetor_resultante = np.zeros((1,300))  
  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn) # Para cada palavra vetorizada, soma nas frases

    except KeyError:
      pass
    
  return vetor_resultante

In [7]:
vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
vetor_texto

array([[-1.10741216,  0.83379558, -0.82869156, -0.02523934, -0.02722143,
        -0.82423201, -0.96666209,  0.36905223, -1.69396117, -0.30547112,
        -0.76654428,  0.10133891,  0.23117292, -0.68575152, -1.09992392,
        -1.44090445,  0.87377934,  0.30984993, -1.90884149,  0.32372817,
        -0.48346579, -1.33966532, -0.73666604, -1.34208505, -1.08936587,
        -1.98750328,  0.89234112,  0.94890755,  0.73362353, -0.26530629,
         1.61073157,  0.18308221, -0.56207922, -0.8248736 ,  0.22283812,
        -0.63860723, -1.14239901,  1.05226414,  1.46034321,  0.494352  ,
         1.02752678,  0.2456468 , -2.57297491, -0.67364249, -0.46491681,
         0.1569852 ,  0.75104317,  0.25027266,  1.35288861,  0.4447321 ,
         1.36689436,  0.82575934, -0.9277437 , -0.54219482, -0.55270942,
        -0.95775029,  0.97733816, -0.58829374, -1.41687343, -1.16170138,
        -1.63575691, -0.2971549 ,  2.34820962, -0.14121422, -0.25960259,
         0.27507701, -0.11019109,  1.76308602,  0.5

In [8]:
# Definindo a função que vai transformar todas as frases em uma matriz
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i]) # Transforma cada linha de dado em uma lista de palavras
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo) # coloca cada frase vetor em sua devida linha da matriz

  return matriz 

In [9]:
# transformando os dados de treino e teste
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)

matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)


In [10]:
print("tamanho treino = ", matriz_vetores_treino_cbow.shape)
print("tamanho teste = ", matriz_vetores_teste_cbow.shape)

tamanho treino =  (90000, 300)
tamanho teste =  (20513, 300)


## Iniciando a Classificação

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [12]:
def classificador(modelo, x_train, y_train, x_test, y_test):
  LR = LogisticRegression(max_iter = 800)
  LR.fit(x_train, y_train)
  categories =  LR.predict(x_test)
  resultados = classification_report(y_test,  categories)
  print(resultados)

  return LR


In [13]:
LR_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow, 
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.75      0.83      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [16]:
# salvando o modelo 
import pickle

with open("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/LR_cbow.pkl", "wb") as f:
  pickle.dump(LR_cbow, f)

In [17]:
# É possível também realizar a classificação utilizando a arquitetura skipgram

# matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
# matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

# LR_sg = classificador(w2v_modelo_sg,
#                       matriz_vetores_treino_sg,
#                       artigo_treino.category,
#                       matriz_vetores_teste_sg,
#                       artigo_teste.category)

# with open("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/LR_sg.pkl", "wb") as f:
#   pickle.dump(LR_sg, f)